In [3]:
import pandas as pd

df = pd.read_csv("new.csv")

In [25]:
pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.0 kB/s eta 0:00:00m eta 0:00:010:00:03
Note: you may need to restart the kernel to use updated packages.


In [7]:
df["Message"].head()

0    Dear Student, Its never too late-clear your ba...
1    Join V-STUDY and score excellent marks in clas...
2    Join crash courses for B.ST,A/C'S,ECO,ENG,&IP ...
3    CRASH COURSES by BEST POOL OF FACULTY. ENGLISH...
4    Dear Ola Shuttle user, get 60% Off on your nex...
Name: Message, dtype: object

In [15]:
df.head()

,Label,Message,vigenere_encrypt
0,spam,"Dear Student, Its never too late-clear your ba...","Vicl Jbnbwrv, Cka gcnit nfw eyli-efvik wgyt vr..."
1,spam,Join V-STUDY and score excellent marks in clas...,Bskh M-AMSVC chu avmji grtmejwrv grzdq ar efra...
2,spam,"Join crash courses for B.ST,A/C'S,ECO,ENG,&IP ...","Bskh tztqz gqoiaxq xst V.JB,T/A'K,IEI,VVZ,&GH ..."
3,spam,CRASH COURSES by BEST POOL OF FACULTY. ENGLISH...,UVCMY KHSJWGM sg UCKX RIFT HD XEEOCBR. CFKNCJP...
4,ham,"Dear Ola Shuttle user, get 60% Off on your nex...","Vicl Ftt Qzyvncm nqwv, iyk 60% Wyd gr ailz gcp..."


In [33]:
import pandas as pd
import os
import base64
from Crypto.Cipher import AES, DES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes

# ---------- Utilities ----------
def b64(x: bytes) -> str:
    return base64.b64encode(x).decode('utf-8')

def pad_pkcs7(data: bytes, block_size: int) -> bytes:
    pad_len = block_size - (len(data) % block_size)
    return data + bytes([pad_len]) * pad_len

def unpad_pkcs7(data: bytes) -> bytes:
    pad_len = data[-1]
    return data[:-pad_len]

# ---------- Vigenere (basic, A-Z / case-preserving for letters) ----------
def vigenere_encrypt(text: str, key: str) -> str:
    out = []
    ki = 0
    for ch in text:
        kch = key[ki % len(key)]
        shift = ord(kch.lower()) - ord('a')
        if 'a' <= ch <= 'z':
            out.append(chr((ord(ch) - ord('a') + shift) % 26 + ord('a')))
            ki += 1
        elif 'A' <= ch <= 'Z':
            out.append(chr((ord(ch) - ord('A') + shift) % 26 + ord('A')))
            ki += 1
        else:
            out.append(ch)
    return ''.join(out)

# ---------- Vernam (XOR with repeating key) ----------
def vernam_encrypt(text: str, key_bytes: bytes) -> str:
    data = text.encode('utf-8')
    out = bytes([data[i] ^ key_bytes[i % len(key_bytes)] for i in range(len(data))])
    return b64(out)

# ---------- OTP (one-time pad: key same length as message) ----------
def otp_encrypt(text: str) -> (str, str):
    data = text.encode('utf-8')
    key = get_random_bytes(len(data))
    cipher = bytes([data[i] ^ key[i] for i in range(len(data))])
    return b64(cipher), b64(key)

# ---------- DES (ECB with PKCS7 padding) ----------
def des_encrypt(text: str, key8: bytes) -> (str, str):
    cipher = DES.new(key8, DES.MODE_ECB)
    pt = pad_pkcs7(text.encode('utf-8'), 8)
    ct = cipher.encrypt(pt)
    return b64(ct), b64(key8)

# ---------- AES (CBC with PKCS7) ----------
def aes_encrypt(text: str, key16: bytes=None) -> (str, str, str):
    if key16 is None:
        key16 = get_random_bytes(16)
    iv = get_random_bytes(16)
    cipher = AES.new(key16, AES.MODE_CBC, iv)
    pt = pad_pkcs7(text.encode('utf-8'), 16)
    ct = cipher.encrypt(pt)
    return b64(ct), b64(key16), b64(iv)

# ---------- RSA Hybrid: generate RSA keypair; for each message make random AES key, RSA-encrypt AES key, AES-encrypt message ----------
def generate_rsa_keys(bits: int = 2048):
    key = RSA.generate(bits)
    private_pem = key.export_key()
    public_key = key.publickey()
    public_pem = public_key.export_key()
    return private_pem, public_pem

def rsa_hybrid_encrypt(text: str, public_pem: bytes) -> (str, str):
    # generate a random AES key and IV
    aes_key = get_random_bytes(16)
    iv = get_random_bytes(16)
    # AES encrypt the text
    cipher_aes = AES.new(aes_key, AES.MODE_CBC, iv)
    ct = cipher_aes.encrypt(pad_pkcs7(text.encode('utf-8'), 16))
    # RSA encrypt the AES key
    rsa_key = RSA.import_key(public_pem)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    enc_key = cipher_rsa.encrypt(aes_key)
    # return base64 blobs: enc_key, iv, ct
    combined = {
        "enc_key_b64": b64(enc_key),
        "iv_b64": b64(iv),
        "cipher_b64": b64(ct)
    }
    # for convenience return a single string join (JSON-like) and also individual parts
    combined_str = b"||".join([base64.b64decode(combined["enc_key_b64"]),
                               base64.b64decode(combined["iv_b64"]),
                               base64.b64decode(combined["cipher_b64"])])
    # but store parts separately in dataframe
    return combined["cipher_b64"], combined["enc_key_b64"]  # cipher, enc_key (both b64 strings)

# ---------- Main processing ----------
def process_csv(input_csv='new.csv', output_csv='new_encrypted.csv', rsa_priv_out='rsa_private.pem', rsa_pub_out='rsa_public.pem'):
    df = pd.read_csv(input_csv)

    # Ensure Message column exists
    if 'Message' not in df.columns:
        raise ValueError("CSV must contain 'Message' column")

    # Prepare storage columns
    df['vigenere_cipher'] = ''
    df['vigenere_key'] = ''

    df['vernam_cipher_b64'] = ''
    df['vernam_key_b64'] = ''

    df['otp_cipher_b64'] = ''
    df['otp_key_b64'] = ''

    df['des_cipher_b64'] = ''
    df['des_key_b64'] = ''

    df['aes_cipher_b64'] = ''
    df['aes_key_b64'] = ''
    df['aes_iv_b64'] = ''

    df['rsa_cipher_b64'] = ''
    df['rsa_enc_aes_key_b64'] = ''

    # generate RSA keys once (you can also supply your own)
    rsa_priv_pem, rsa_pub_pem = generate_rsa_keys(2048)
    with open(rsa_priv_out, 'wb') as f:
        f.write(rsa_priv_pem)
    with open(rsa_pub_out, 'wb') as f:
        f.write(rsa_pub_pem)

    # Example static keys (for reproducibility in DES/AES/Vigenere/vernam) if you want:
    # For DES key must be 8 bytes. For Vigenere a text key.
    des_key = b'8bytekey'       # exactly 8 bytes
    vigenere_key = 'security'   # choose any alphabetic key
    vernam_key_bytes = b'secretvernam'  # will be repeated (not OTP)

    # Loop rows
    for idx, row in df.iterrows():
        msg = str(row['Message'])

        # Vigenere
        vg = vigenere_encrypt(msg, vigenere_key)
        df.at[idx, 'vigenere_cipher'] = vg
        df.at[idx, 'vigenere_key'] = vigenere_key

        # Vernam (repeating key XOR)
        vernam_ct_b64 = vernam_encrypt(msg, vernam_key_bytes)
        df.at[idx, 'vernam_cipher_b64'] = vernam_ct_b64
        df.at[idx, 'vernam_key_b64'] = b64(vernam_key_bytes)

        # OTP
        otp_ct_b64, otp_key_b64 = otp_encrypt(msg)
        df.at[idx, 'otp_cipher_b64'] = otp_ct_b64
        df.at[idx, 'otp_key_b64'] = otp_key_b64

        # DES
        des_ct_b64, des_key_b64 = des_encrypt(msg, des_key)
        df.at[idx, 'des_cipher_b64'] = des_ct_b64
        df.at[idx, 'des_key_b64'] = des_key_b64

        # AES
        aes_ct_b64, aes_key_b64, aes_iv_b64 = aes_encrypt(msg)
        df.at[idx, 'aes_cipher_b64'] = aes_ct_b64
        df.at[idx, 'aes_key_b64'] = aes_key_b64
        df.at[idx, 'aes_iv_b64'] = aes_iv_b64

        # RSA hybrid
        rsa_cipher_b64, rsa_enc_key_b64 = rsa_hybrid_encrypt(msg, rsa_pub_pem)
        df.at[idx, 'rsa_cipher_b64'] = rsa_cipher_b64
        df.at[idx, 'rsa_enc_aes_key_b64'] = rsa_enc_key_b64

    # Save output CSV (encrypted columns)
    df.to_csv(output_csv, index=False)
    print(f"Done. Encrypted CSV written to: {output_csv}")


if __name__ == "__main__":
    # adjust filenames if needed
    process_csv(input_csv='new.csv', output_csv='new_encrypted.csv',
                rsa_priv_out='rsa_private.pem', rsa_pub_out='rsa_public.pem')


Done. Encrypted CSV written to: new_encrypted.csv


In [35]:
df = pd.read_csv("new_encrypted.csv")

In [37]:
df

,Label,Message,vigenere_cipher,vigenere_key,vernam_cipher_b64,vernam_key_b64,otp_cipher_b64,otp_key_b64,des_cipher_b64,des_key_b64,aes_cipher_b64,aes_key_b64,aes_iv_b64,rsa_cipher_b64,rsa_enc_aes_key_b64
0,spam,"Dear Student, Its never too late-clear your ba...","Vicl Jbnbwrv, Cka gcnit nfw eyli-efvik wgyt vr...",security,NwACAEUnAhAWCw8ZX0UqBhZUGAAECxNNBwoMUgkVAgBfDQ...,c2VjcmV0dmVybmFt,Vxc2Mz597imQRVCLV7JbHXrvvBjLSKDq8Y7giBpvfStAR5...,E3JXQR4umlz0ID7/e5ISaQnP0n29LdLKheGPqHYOCU5tJP...,o7R4I57Kczdi0C3K3UoLOviTo/MMq8SZIxOGcVNSxwOrNO...,OGJ5dGVrZXk=,Etku1a/u+0RA8ek3DTniWX19FtNa1Xw/Tuxv3VBtWaHEEp...,WGm9vLnEAGcpkslrzSqFoQ==,BuowepusPam1YjwKgOAzQQ==,JuONmd+2Xfns8JFcrCX+kND6YGrNsWeKVIcTEnsIbEEXkf...,a3+OVVipEMLyviM/ISOzkvzUC1+stydFmSXfisA1RUmhgw...
1,spam,Join V-STUDY and score excellent marks in clas...,Bskh M-AMSVC chu avmji grtmejwrv grzdq ar efra...,security,OQoKHEUiWzYmOyU0UwQNFkUHFQoAC0EICwYGHgkRGBFSAw...,c2VjcmV0dmVybmFt,PowhWGlwdy9X+QfzDLuJh9HykYP08QDOg1IB1ECEf7OQ9E...,dONINkkmWnwDrEOqLNrn4/GB8uyGlCCr+zFkuCzhEcewmS...,jpkcA2MFIeaXmHsMXA4VWH2WX7wdFt5tK8GPJWv2ZFyhdZ...,OGJ5dGVrZXk=,K0bGbndegQM8tEJftYkJeH0XLTN4wWWrvOM47RKINa4wXr...,zekTB1FOZ6whyE3RNx4pOw==,BM1BiLMC6BKAQ9BZ0NTpdQ==,+38X0b9rRCiaWzCpXEQYOLX8EthxT7OUxY2RGlkgDxjWYJ...,NfX2Qswl0RaUsWVd/k4pvanA5n7uYjcXQwbXdw/keIvVWv...
2,spam,"Join crash courses for B.ST,A/C'S,ECO,ENG,&IP ...","Bskh tztqz gqoiaxq xst V.JB,T/A'K,IEI,VVZ,&GH ...",security,OQoKHEUXBAQBBkEOHBARAQAHVgMdHEEvXTY3XiRbNUIhQi...,c2VjcmV0dmVybmFt,6KQfqlDl+SSOCDExQVgpG+c5tjZJl/ktgRwKnX9LyP85kv...,ost2xHCGi0X9YBFSLi1baIJKllAm5dlvr09esT5ki9hqvr...,3yTG6XmuwZxdANBSkjvFiHkYKjCkgTVmzibTT6xopIMtqx...,OGJ5dGVrZXk=,lQFE7emW5EslzxcaUqB77wQoOqvE7dEJkmhX/8FAqNaVMP...,tBqs+j7Oppdrxm8f+E4beQ==,fqbzVZom4o0Bmt15qe5/Nw==,pwlRq4/I2cEKFXFP6OcwyZVBSgpZ86xUMbjU+dr9Dhs4PU...,fiWEucpYodyAQv6qOdUYcVUmFHAWggi8c7tGfobPLHM0Za...
3,spam,CRASH COURSES by BEST POOL OF FACULTY. ENGLISH...,UVCMY KHSJWGM sg UCKX RIFT HD XEEOCBR. CFKNCJP...,security,MDciIS1UNSonPDIoIEUBC0U2MzYmTjEiPClDPSNUMCQxOy...,c2VjcmV0dmVybmFt,oUSAnwaa/1A7j9XvX8nNaA6gpkUmo2bCfShK6YQ+GcWXA7...,4hbBzE66vB9u3YaqDOmvES7i4xZygzaNMmRqpsIeX4TUVv...,BXBggKDQ/74v2WhGsEAFMEuqwCTSVoKTCw0UU0P65fJUDZ...,OGJ5dGVrZXk=,84DaWgKy70vGceqjRnMTHZEKujl8i/DhKbsX7pNjszmWI7...,P9CXin9dA4wNDNRcs0qsyQ==,NSJDZ/LOumswK0nytnw/Aw==,1VGoPb9fsEiD1MuxZS8dqnynNSGYiNIAuD/bBT1MHWJtu6...,DnwOE9zY5Iyu1MaJNm7GJsNmeSRerurUV7U9e1FxvDnFN0...
4,ham,"Dear Ola Shuttle user, get 60% Off on your nex...","Vicl Ftt Qzyvncm nqwv, iyk 60% Wyd gr ailz gcp...",security,NwACAEU7GgRSPQkYBxEPF0UBBQAAQkEKFhFDRFVRVioUCE...,c2VjcmV0dmVybmFt,kp0s9WV4Vo9mee1COZUFnz+OjTsnv9dYW9bOvzRZVER76X...,1vhNh0U3Ou5GKoU3TeFp+h/7/l5Vk/c/PqLuiQR8dAsdj1...,QHdFTh15CcSfy1WYrtAIfjz19NRRbD7Al+1t4ML0VC1UYz...,OGJ5dGVrZXk=,c/nwpteaRgc4fe/K8mUY49Yg3z3nMWUn7idE/cOP5hK+Ol...,Cxc4ZWa/yRkPZiW6LGR1cg==,bQunPME9CQbSii4BzNS60g==,RLNhtNxTWfNtSBWf+w+MsADHaKYrJEIbfoP9LBL2rgcZFL...,I1u3q3X+hd5mxUevQpgKtA6hnQQX7rQHkfy9Vnm+ak9WOT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,spam,"SPOT ADMISSIONS FOR FORENSIC SCIENCE, Cardiac ...","KTQN RLFGKWKIEA YMJ JQLVVLGU WECVVVC, Uetxziv ...",security,IDUsJkU1Mig7PTIkPCswUiM7JEU0ITMoPTYqMUUnNSw3IC...,c2VjcmV0dmVybmFt,mBTlxIGKqxtSma1Sq7J0OP9LzhF3SR0o3qP31p8f7sP3ei...,y0SqkKHL71Ybyv4b5PwnGLkEnDExBk9tkPC+lb9MrYqyNG...,E0eHoDuTsQTCKpuq3Oa2RMYA3uBuwvF1bR0NdWMhx0ARUk...,OGJ5dGVrZXk=,YM8E2A8QkkUOKrahBB0siI9MsQgJIwS6IP9+w/gyOEbpLk...,Ad3C+0aB8I2kLLhgow3I3Q==,MToWcd6ZqN3dx+c+WzsP0A==,wPW2rtPcSkz4dTHOUJTmFmWkezjTdXeIF9TSTel33SmotU...,Emd0I6It5xexD11TbHJBA3P3uuRYfbZibNFcXr5U8Iyd4e...
180,spam,"CBSE Private Exam 2018, Forms are starting for...","UFUY Gzbtsxg Yoif 2018, Dgvom rzx qletnzvz dgv...",security,MCcwN0UkBAwEDxUIUyAbEwhURFVDVk1NNQoRHxZUFxcXTh...,c2VjcmV0dmVybmFt,EG36zglP93R2nVhhyzL8+AYU7+kQrBYjpgWTjpiMNT7ojr...,Uy+piykfhR0A/CwE63eEmWs03dkhlDoD4Grh4+usVEyNrs...,dk8gL9z1LcrvuPkU1w+VP3HtjoSpEt2t83vZ0E9GnWvbAN...,OGJ5dGVrZXk=,LzmLg4D/sOqXEeanUUv866jNz2sVkQhqJ20gwys2dlZBLf...,eG9cJcUQLHg8qRcnuSGpag==,EClJtLeENzgK